In [2]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

# Package imports
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model

In [3]:
import h5py
def save_model(model):
#     a = np.random.random(size=(100,20))
#     h5f.close()
    h5f = h5py.File('data.h5', 'w')
    adict=dict(W1=model['W1'], b1=model['b1'], W2=model['W2'], b2=model['b2'])
    for k,v in adict.items():
        h5f.create_dataset(k,data=v)

#     h5f.create_dataset('dataset_1', data=model)
    h5f.close()

def load_model():
    h5f = h5py.File('data.h5','r')
    model = {}
    model['W1'] = h5f['W1'][:]
    model['b1'] = h5f['b1'][:]
    model['W2'] = h5f['W2'][:]
    model['b2'] = h5f['b2'][:]
    h5f.close()
    return model

In [43]:
h5f = h5py.File('data.h5','r')
b = h5f['dataset_1'][:]
h5f.close()
print b

[[ 0.74477917  0.74176513  0.2249075  ...,  0.78764643  0.00854378
   0.13474878]
 [ 0.8705172   0.33618638  0.87997369 ...,  0.26633011  0.44902558
   0.80370298]
 [ 0.64539694  0.01935548  0.66401021 ...,  0.39910207  0.90031777
   0.11416035]
 ..., 
 [ 0.24709955  0.84189366  0.75455214 ...,  0.06533378  0.41221683
   0.78579633]
 [ 0.24941092  0.3949953   0.86314813 ...,  0.19445702  0.94075823
   0.93692826]
 [ 0.59613279  0.98433597  0.68316126 ...,  0.98291398  0.3052608
   0.8792701 ]]


In [8]:
with open('../../data/SlovarIJS_BESEDE_utf8.lex') as f:
    content = f.readlines()
content = [x.decode('utf8').split('\t') for x in content]

In [9]:
# CREATE dictionary AND max_word
accetuated_vowels = [u'à', u'á', u'ä', u'é', u'ë', u'ì', u'í', u'î', u'ó', u'ô', u'ö', u'ú', u'ü']
default_vowels = [u'a', u'e', u'i', u'o', u'u']
vowels = []
vowels.extend(accetuated_vowels)
vowels.extend(default_vowels)
def is_vowel(word_list, position):
    if word_list[position] in vowels:
        return True
    if word_list[position] == u'r' and \
    (position - 1 < 0 or word_list[position - 1] not in vowels) and \
    (position + 1 >= len(word_list) or word_list[position + 1] not in vowels):
        return True
    return False
        
    

dictionary = ['']
line = 0
max_word = 0
# ADD 'EMPTY' VOWEL
max_num_vowels = 0
for el in content:
    num_vowels = 0
    i = 0
    try: 
        if len(el[3]) > max_word:
            max_word = len(el[3])
#             print line
        if len(el[0]) > max_word:
            max_word = len(el[0])
            print line
        for c in list(el[3]):
            if is_vowel(list(el[3]), i):
                num_vowels += 1
            if c not in dictionary:
                dictionary.append(c)
            i += 1
        for c in list(el[0]):
            if c not in dictionary:
                dictionary.append(c)
        if num_vowels > max_num_vowels:
            max_num_vowels = num_vowels
#             print line
    except Exception, e:
        print line - 1
        print el
    line += 1
dictionary = sorted(dictionary)
max_num_vowels += 1

# for e in dictionary:
#     print e

In [12]:
# GENERATE X and y
def generate_presentable_y(accetuations_list, word_list):
#     print accetuations_list
#     print word_list
    while len(accetuations_list) < 2:
        accetuations_list.append(0)
    if len(accetuations_list) > 2:
        accetuations_list = accetuations_list[:2]
#     empty has to be positive
    accetuations_list = np.array(accetuations_list)
#     accetuations_list += 1
    final_position = accetuations_list[0] + max_num_vowels * accetuations_list[1]
    return final_position
    

X = np.zeros((len(content), max_word*len(dictionary)))
y = np.zeros((len(content), max_num_vowels * max_num_vowels ))


i = 0
for el in content:
    j = 0
    for c in list(el[0]):
        index = 0
        for d in dictionary:
            if c == d:
                X[i][index + j * max_word] = 1
                break
            index += 1
        j += 1
    j = 0
    word_accetuations = []
    num_vowels = 0
    for c in list(el[3]):
        index = 0
#         in_dict = False
        if is_vowel(el[3], j):
            num_vowels += 1
        for d in accetuated_vowels:
#             print c
#             print d
            if c == d:
#                 print 'HERE!'
                word_accetuations.append(num_vowels)
#                 y[j * len(dictionary) + index][i] = 1
#                 in_dict = True
                break
            index += 1
#         if not in_dict:
#             print 'NOOOOO!!!'
#             print el
#             print i
        j += 1
#     print len(y.T[i])
#     print sum(y.T[i])
    
#     y = y.T
    y[i][generate_presentable_y(word_accetuations, list(el[3]))] = 1
#     print '---------------------------------'
#     for a in y[i]:
#         print a
#     print sum(y[i])
#     print y[:][i]
#     if i > 100:
#         break
    i += 1


In [13]:
def decode_position(y):
    max_el = 0
    i = 0
    pos = -1
#     pos = y.argmax(axis=0)
    for el in y:
#         print el
        if el > max_el:
            
            max_el = el
#             print 'HERE!!!'
#             print pos
            pos = i
        i += 1
    return [pos % max_num_vowels, pos / max_num_vowels]
    
decode_position(y[157303])
# decode_position(y[157241])
# print y[2]

[1, 3]

In [29]:
X = X[:100000]
y = y[:100000]
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
X, y = unison_shuffled_copies(X, y)
# print X.shape

num_examples = len(X) # training set size
nn_input_dim = max_word * len(dictionary) # input layer dimensionality
nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = 0.00001 # learning rate for gradient descent
reg_lambda = 0.00001 # regularization strength

In [25]:
print X[0]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [47]:

# Helper function to evaluate the total loss on the dataset
def calculate_loss(model):
    save_model(model)
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation to calculate our predictions
    z1 = X.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
#     print exp_scores
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
#     print exp_score
    # Calculating the loss
#     print probs.shape
#     print y.shape
#     return
#     print 'IS THIS SLOW?'
    corect_logprobs = -np.log(np.sum(probs * y, axis=1))
#     print corect_logprobs.shape
#     print corect_logprobs[:10]
#     print 'CALC SUM'
    data_loss = np.sum(corect_logprobs)
#     print 'HERE'
#     print data_loss
    # Add regulatization term to loss (optional)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss

In [5]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
#     print probs
    return probs
#     return np.argmax(probs, axis=1)

In [1]:
# This function learns parameters for the neural network and returns the model.
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(nn_hdim, num_passes=20000, print_loss=False):
    
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))

    # This is what we return at the end
    model = {}
    
    # Gradient descent. For each batch...
    for i in xrange(0, num_passes):
        print 'NUM: ' + str(i)
        # Forward propagation
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta3 = probs
#         delta3[range(num_examples), y] -= 1
#         print probs
#         print y
        delta3 = probs - y
#         print delta3
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # Add regularization terms (b1 and b2 don't have regularization terms)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1

        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
#         print model
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        test = calculate_loss(model)
        if print_loss and i % 25 == 0:
          print "Loss after iteration %i: %f" %(i, calculate_loss(model))
#           return None
    
    return model

In [55]:
# ACCETUATION DATA
# Build a model with a 3-dimensional hidden layer
model = build_model(500, print_loss=True)

NUM: 0
Loss after iteration 0: 3.880257
NUM: 1
NUM: 2
NUM: 3
NUM: 4
NUM: 5
NUM: 6
NUM: 7
NUM: 8
NUM: 9
NUM: 10
NUM: 11
NUM: 12
NUM: 13
NUM: 14
NUM: 15
NUM: 16
NUM: 17
NUM: 18
NUM: 19
NUM: 20
NUM: 21
NUM: 22
NUM: 23
NUM: 24
NUM: 25
Loss after iteration 25: 1.218822
NUM: 26
NUM: 27
NUM: 28
NUM: 29
NUM: 30
NUM: 31
NUM: 32
NUM: 33
NUM: 34
NUM: 35
NUM: 36
NUM: 37
NUM: 38
NUM: 39
NUM: 40
NUM: 41
NUM: 42
NUM: 43
NUM: 44
NUM: 45
NUM: 46
NUM: 47
NUM: 48
NUM: 49
NUM: 50
Loss after iteration 50: 0.894443
NUM: 51
NUM: 52
NUM: 53
NUM: 54
NUM: 55
NUM: 56
NUM: 57
NUM: 58
NUM: 59
NUM: 60
NUM: 61
NUM: 62
NUM: 63
NUM: 64
NUM: 65
NUM: 66
NUM: 67
NUM: 68
NUM: 69
NUM: 70
NUM: 71
NUM: 72
NUM: 73
NUM: 74
NUM: 75
Loss after iteration 75: 0.797873
NUM: 76
NUM: 77
NUM: 78
NUM: 79
NUM: 80
NUM: 81
NUM: 82
NUM: 83
NUM: 84
NUM: 85
NUM: 86
NUM: 87
NUM: 88
NUM: 89
NUM: 90
NUM: 91
NUM: 92
NUM: 93
NUM: 94
NUM: 95
NUM: 96
NUM: 97
NUM: 98
NUM: 99
NUM: 100
Loss after iteration 100: 0.739116
NUM: 101
NUM: 102
NUM: 103
NUM:

KeyboardInterrupt: 

In [16]:
def generate_input_from_word(word):
    x = np.zeros(max_word*len(dictionary))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[index + j * max_word] = 1
                break
            index += 1
        j += 1
    return x

model = load_model()
prediction = predict(model, generate_input_from_word('hidrija'))
print decode_position(prediction[0])

[2, 0]
